In [1]:
from data_loader import *
from utils import *

from concurrent.futures import ThreadPoolExecutor,as_completed
from itertools import product

In [2]:
leagues_dat = get_leagues(home_dir +"/data/Leagues/leagues.parquet")
leagues_dat[['league_id','league_name','country_name']].head()

,league_id,league_name,country_name
0,4,Euro Championship,World
1,21,Confederations Cup,World
2,61,Ligue 1,France
3,144,Jupiler Pro League,Belgium
4,71,Serie A,Brazil


In [11]:
teams_data.head()

,team_name,league
0,Angers,61
1,Lille,61
2,Lyon,61
3,Marseille,61
4,Nantes,61


In [3]:
teams_data = pd.read_parquet(home_dir + "/data/Teams/team_league.parquet")
teams_data = teams_data.merge(leagues_dat,left_on='league',right_on="league_id",how = "left")
teams_data.head()


,team_name,league,seasons,league_id,league_name,league_type,league_logo,country_name,country_code,country_flag
0,Angers,61,"[{'coverage': {'fixtures': {'events': True, 'l...",61,Ligue 1,League,https://media.api-sports.io/football/leagues/6...,France,FR,https://media.api-sports.io/flags/fr.svg
1,Lille,61,"[{'coverage': {'fixtures': {'events': True, 'l...",61,Ligue 1,League,https://media.api-sports.io/football/leagues/6...,France,FR,https://media.api-sports.io/flags/fr.svg
2,Lyon,61,"[{'coverage': {'fixtures': {'events': True, 'l...",61,Ligue 1,League,https://media.api-sports.io/football/leagues/6...,France,FR,https://media.api-sports.io/flags/fr.svg
3,Marseille,61,"[{'coverage': {'fixtures': {'events': True, 'l...",61,Ligue 1,League,https://media.api-sports.io/football/leagues/6...,France,FR,https://media.api-sports.io/flags/fr.svg
4,Nantes,61,"[{'coverage': {'fixtures': {'events': True, 'l...",61,Ligue 1,League,https://media.api-sports.io/football/leagues/6...,France,FR,https://media.api-sports.io/flags/fr.svg


In [4]:
teams_data[teams_data.league_name.str.contains("Premier League")]

,team_name,league,seasons,league_id,league_name,league_type,league_logo,country_name,country_code,country_flag
38,Manchester United,39,"[{'coverage': {'fixtures': {'events': True, 'l...",39,Premier League,League,https://media.api-sports.io/football/leagues/3...,England,GB,https://media.api-sports.io/flags/gb.svg
39,Newcastle,39,"[{'coverage': {'fixtures': {'events': True, 'l...",39,Premier League,League,https://media.api-sports.io/football/leagues/3...,England,GB,https://media.api-sports.io/flags/gb.svg
40,Bournemouth,39,"[{'coverage': {'fixtures': {'events': True, 'l...",39,Premier League,League,https://media.api-sports.io/football/leagues/3...,England,GB,https://media.api-sports.io/flags/gb.svg
41,Fulham,39,"[{'coverage': {'fixtures': {'events': True, 'l...",39,Premier League,League,https://media.api-sports.io/football/leagues/3...,England,GB,https://media.api-sports.io/flags/gb.svg
42,Wolves,39,"[{'coverage': {'fixtures': {'events': True, 'l...",39,Premier League,League,https://media.api-sports.io/football/leagues/3...,England,GB,https://media.api-sports.io/flags/gb.svg
...,...,...,...,...,...,...,...,...,...,...
761,Crystal Palace,39,"[{'coverage': {'fixtures': {'events': True, 'l...",39,Premier League,League,https://media.api-sports.io/football/leagues/3...,England,GB,https://media.api-sports.io/flags/gb.svg
762,West Brom,39,"[{'coverage': {'fixtures': {'events': True, 'l...",39,Premier League,League,https://media.api-sports.io/football/leagues/3...,England,GB,https://media.api-sports.io/flags/gb.svg
763,Sheffield Utd,39,"[{'coverage': {'fixtures': {'events': True, 'l...",39,Premier League,League,https://media.api-sports.io/football/leagues/3...,England,GB,https://media.api-sports.io/flags/gb.svg
764,Leeds,39,"[{'coverage': {'fixtures': {'events': True, 'l...",39,Premier League,League,https://media.api-sports.io/football/leagues/3...,England,GB,https://media.api-sports.io/flags/gb.svg


In [8]:
# Pr
read_all_teams = False
unique_teams = teams_data[teams_data.league_id == 39]['team_name'].unique()

if read_all_teams:
    # All teams:
    unique_teams = teams_data['team_name'].unique()

print(f"Number of Teams to pull: {len(unique_teams)} ")
# Specific teams:
print("Expected API requests: ",int(len(unique_teams) * 38 * .8))

Number of Teams to pull: 28 
Expected API requests:  851


In [9]:
teams = unique_teams
season = [2025]
team_season_pair = list(product(teams,season))

In [10]:
team_season_pair

[('Manchester United', 2025),
 ('Newcastle', 2025),
 ('Bournemouth', 2025),
 ('Fulham', 2025),
 ('Wolves', 2025),
 ('Liverpool', 2025),
 ('Arsenal', 2025),
 ('Burnley', 2025),
 ('Everton', 2025),
 ('Tottenham', 2025),
 ('West Ham', 2025),
 ('Chelsea', 2025),
 ('Manchester City', 2025),
 ('Brighton', 2025),
 ('Crystal Palace', 2025),
 ('Brentford', 2025),
 ('Leeds', 2025),
 ('Nottingham Forest', 2025),
 ('Aston Villa', 2025),
 ('Sunderland', 2025),
 ('Southampton', 2025),
 ('Leicester', 2025),
 ('Ipswich', 2025),
 ('Sheffield Utd', 2025),
 ('Luton', 2025),
 ('Watford', 2025),
 ('Norwich', 2025),
 ('West Brom', 2025)]

In [ ]:
parallel = False
workers = 3
visited_list = list()

if parallel:
    print(f"Running in parallel with {workers} workers")
    with ThreadPoolExecutor(max_workers=workers) as executor:
        futures = {executor.submit(read_fixtures_for_season,t,s) : (t,s) for t,s in team_season_pair}
        
else:
    for team in teams:
        try:
            dat = read_fixtures_for_season(team,season,sleep_time=60)
            visited_list.append(team)
        except:
            print(f"{team} does not have matches in {season}")

Running in parallel with 3 workers
processing for Angers, 2025
processing for Lille, 2025
processing for Lyon, 2025
Pulling for Lille,2025 with league_id: 61
Pulling for Angers,2025 with league_id: 61
Pulling for Lyon,2025 with league_id: 61


/Users/echhitjoshi/Library/Mobile Documents/com~apple~CloudDocs/Work/overperformXG/app/data_loader.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_fixtures['winner'] = np.where(team_fixtures.teams_home_winner == True,team_fixtures.teams_home_name,
/Users/echhitjoshi/Library/Mobile Documents/com~apple~CloudDocs/Work/overperformXG/app/data_loader.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_fixtures['winner'] = np.where(team_fixtures.teams_home_winner == True,team_fixtures.teams_home_na

processing for Marseille, 2025
Pulling for Marseille,2025 with league_id: 61


/Users/echhitjoshi/Library/Mobile Documents/com~apple~CloudDocs/Work/overperformXG/app/data_loader.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_fixtures['winner'] = np.where(team_fixtures.teams_home_winner == True,team_fixtures.teams_home_name,


processing for Nantes, 2025
Pulling for Nantes,2025 with league_id: 61
processing for Nice, 2025
Pulling for Nice,2025 with league_id: 61


...

processing for Montpellier, 2025
Pulling for Montpellier,2025 with league_id: 61
processing for Reims, 2025
Pulling for Reims,2025 with league_id: 61
processing for Saint Etienne, 2025
Pulling for Saint Etienne,2025 with league_id: 61
processing for Atletico Paranaense, 2025
Pulling for Atletico Paranaense,2025 with league_id: 71
processing for Criciuma, 2025
Pulling for Criciuma,2025 with league_id: 71
processing for Atletico Goianiense, 2025
Pulling for Atletico Goianiense,2025 with league_id: 71
processing for Cuiaba, 2025
Pulling for Cuiaba,2025 with league_id: 71


/Users/echhitjoshi/Library/Mobile Documents/com~apple~CloudDocs/Work/overperformXG/app/data_loader.py:113: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fixture_dat_expanded['team_non_penalty_goals_scored'] = fixtures[(fixtures.fixture_id == fixture)]['goals_home'].values[0] - fixtures[(fixtures.fixture_id == fixture)]['score_penalty_home'].fillna(0).values[0]
/Users/echhitjoshi/Library/Mobile Documents/com~apple~CloudDocs/Work/overperformXG/app/data_loader.py:116: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fixture_dat_expanded['team_non_penalty_goals

processing for Southampton, 2025
Pulling for Southampton,2025 with league_id: 39
processing for Leicester, 2025
Pulling for Leicester,2025 with league_id: 39
processing for Ipswich, 2025
Pulling for Ipswich,2025 with league_id: 39
processing for VfL Bochum, 2025
Pulling for VfL Bochum,2025 with league_id: 78
processing for Holstein Kiel, 2025
Pulling for Holstein Kiel,2025 with league_id: 78
processing for SV Elversberg, 2025
Pulling for SV Elversberg,2025 with league_id: 78
processing for Empoli, 2025
Pulling for Empoli,2025 with league_id: 135
processing for Venezia, 2025
Pulling for Venezia,2025 with league_id: 135


/Users/echhitjoshi/Library/Mobile Documents/com~apple~CloudDocs/Work/overperformXG/app/data_loader.py:113: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fixture_dat_expanded['team_non_penalty_goals_scored'] = fixtures[(fixtures.fixture_id == fixture)]['goals_home'].values[0] - fixtures[(fixtures.fixture_id == fixture)]['score_penalty_home'].fillna(0).values[0]
/Users/echhitjoshi/Library/Mobile Documents/com~apple~CloudDocs/Work/overperformXG/app/data_loader.py:116: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fixture_dat_expanded['team_non_penalty_goals

processing for Monza, 2025
Pulling for Monza,2025 with league_id: 135
processing for Willem II, 2025
Pulling for Willem II,2025 with league_id: 88
processing for ADO Den Haag, 2025
Pulling for ADO Den Haag,2025 with league_id: 88
processing for De Graafschap, 2025
Pulling for De Graafschap,2025 with league_id: 88
processing for Dordrecht, 2025
Pulling for Dordrecht,2025 with league_id: 88
processing for Waalwijk, 2025
Pulling for Waalwijk,2025 with league_id: 88
processing for Almere City FC, 2025
processing for Cambuur, 2025
Pulling for Cambuur,2025 with league_id: 88
Pulling for Almere City FC,2025 with league_id: 88
processing for Den Bosch, 2025
Pulling for Den Bosch,2025 with league_id: 88
processing for Las Palmas, 2025
Pulling for Las Palmas,2025 with league_id: 140
processing for Leganes, 2025
Pulling for Leganes,2025 with league_id: 140
processing for Valladolid, 2025
Pulling for Valladolid,2025 with league_id: 140
processing for Clermont Foot, 2025
Pulling for Clermont Foot,2

In [ ]:
teams = list(set(teams) - set(visited_list))
season = [2025]
team_season_pair = list(product(teams,season))

team_season_pair



In [ ]:
len(teams)

In [ ]:
team_season_pair[0:3]